In [60]:
library(data.table)
library(dplyr)
library(tidyr)

In [61]:
path="../DataArtist/instacart"

In [62]:
aisles = fread(file.path(path, "aisles.csv"))
departments=fread(file.path(path,"departments.csv"))
orderp <- fread(file.path(path, "order_products__prior.csv"))
ordert <- fread(file.path(path, "order_products__train.csv"))
orders <- fread(file.path(path, "orders.csv"))
products <- fread(file.path(path, "products.csv"))

Read 3421083 rows and 7 (of 7) columns from 0.101 GB file in 00:00:05


In [63]:
# reshape the data

In [64]:
aisles$aisle <- as.factor(aisles$aisle)
departments$department <- as.factor(departments$department)
orders$eval_set <- as.factor(orders$eval_set)
products$product_name <- as.factor(products$product_name)

# 把products,department和aisles三张表并在一起
products <- products %>% 
  inner_join(aisles) %>% inner_join(departments) %>% 
  select(-aisle_id, -department_id)
rm(aisles, departments)

# 给trainset加了一个user id
ordert$user_id <- orders$user_id[match(ordert$order_id, orders$order_id)]
# 把order表和order_products_prior表以及order_products_train表inner join了一下
# left join是以左表为基础
# inner join没有以谁为基础，一定要两个表有值相等才能连接
orders_products <- orders %>% inner_join(orderp, by = "order_id")

rm(orderp)
gc()

Joining, by = "aisle_id"
Joining, by = "department_id"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,510515,27.3,1875182,100.2,2343978,125.2
Vcells,54553293,416.3,116325960,887.5,121088921,923.9


### 生成关于Products的新特征

In [65]:
prd <- orders_products %>%
  arrange(user_id, order_number, product_id) %>%
  group_by(user_id, product_id) %>%
  mutate(product_time = row_number()) %>%
  ungroup() %>%
  group_by(product_id) %>%
  summarise(
# 该商品被购买的总次数
    prod_orders = n(),
# 该商品被再次购买的总次数
    prod_reorders = sum(reordered),
# 该商品被购买一次的次数
    prod_first_orders = sum(product_time == 1),
# 该商品被购买一次以上的总次数
    prod_second_orders = sum(product_time == 2)
  )

In [66]:
prd <- orders_products %>%
  arrange(user_id, order_number, product_id) %>%
  group_by(user_id, product_id) %>%
  mutate(product_time = row_number()) %>%
  ungroup() %>%
  group_by(product_id) %>%
  summarise(
    prod_orders = n(),
    prod_reorders = sum(reordered),
    prod_first_orders = sum(product_time == 1),
    prod_second_orders = sum(product_time == 2)
  )
# 生成了一个关于products的新表和几个新的特征
# 特征1：products在所有order中出现的次数(该指标含义不清晰)
prd$prod_reorder_probability <- prd$prod_second_orders / prd$prod_first_orders
# 特征2：重购倍数
prd$prod_reorder_times <- 1 + prd$prod_reorders / prd$prod_first_orders
# 特征3：商品被同一用户再次购买的比率
prd$prod_reorder_ratio <- prd$prod_reorders / prd$prod_orders

prd <- prd %>% select(-prod_reorders, -prod_first_orders, -prod_second_orders)

rm(products)
gc()


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,460565,24.6,1465739,78.3,2343978,125.2
Vcells,54241754,413.9,116325960,887.5,121088921,923.9


In [67]:
prd

product_id,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio
1,61,0.05172414,1.724138,0.6885246
2,3,0.00000000,1.000000,0.0000000
3,3,0.00000000,1.666667,0.6666667
4,10,0.11111111,1.333333,0.3000000
8,2,0.00000000,1.000000,0.0000000
9,2,0.00000000,1.500000,0.5000000
10,65,0.05000000,1.516667,0.4769231
12,12,0.09090909,1.636364,0.5833333
14,2,0.00000000,1.000000,0.0000000
15,1,0.00000000,2.000000,1.0000000


### 生成关于用户的新特征

In [53]:
users <- orders %>%
  filter(eval_set == "prior") %>%
  group_by(user_id) %>%
  summarise(
    user_orders = max(order_number),
    user_period = sum(days_since_prior_order, na.rm = T),
    user_mean_days_since_prior = mean(days_since_prior_order, na.rm = T)
  )

us <- orders_products %>%
  group_by(user_id) %>%
  summarise(
    user_total_products = n(),
    user_reorder_ratio = sum(reordered == 1) / sum(order_number > 1),
    user_distinct_products = n_distinct(product_id)
  )

users <- users %>% inner_join(us)
users$user_average_basket <- users$user_total_products / users$user_orders

us <- orders %>%
  filter(eval_set != "prior") %>%
  select(user_id, order_id, eval_set,
         time_since_last_order = days_since_prior_order)

users <- users %>% inner_join(us)

rm(us)
gc()

Joining, by = "user_id"
Joining, by = "user_id"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,460622,24.6,1566652,83.7,2164898,115.7
Vcells,54830236,418.4,121089543,923.9,121088921,923.9


### 建立一个用于分析的Database

In [54]:
data <- orders_products %>%
  group_by(user_id, product_id) %>% 
  summarise(
    up_orders = n(),
    up_first_order = min(order_number),
    up_last_order = max(order_number),
    up_average_cart_position = mean(add_to_cart_order))

rm(orders_products, orders)

data <- data %>% 
  inner_join(prd, by = "product_id") %>%
  inner_join(users, by = "user_id")

data$up_order_rate <- data$up_orders / data$user_orders
data$up_orders_since_last_order <- data$user_orders - data$up_last_order
data$up_order_rate_since_first_order <- data$up_orders / (data$user_orders - data$up_first_order + 1)

data <- data %>% 
  left_join(ordert %>% select(user_id, product_id, reordered), 
            by = c("user_id", "product_id"))

rm(ordert, prd, users)
gc()
  

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,460510,24.6,1875182,100.2,2343978,125.2
Vcells,31186876,238.0,96871634,739.1,121088921,923.9


In [55]:
data

user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,...,user_reorder_ratio,user_distinct_products,user_average_basket,order_id,eval_set,time_since_last_order,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,reordered
4,22199,1,4,4,1.0,2,0.00000000,1.000000,0.0000000,...,0.0000000,2,0.40,329954,test,30,0.20,1,0.50,NA
4,25146,1,4,4,2.0,783,0.09815078,1.716927,0.6436782,...,0.0000000,2,0.40,329954,test,30,0.20,1,0.50,NA
7,4920,1,17,17,6.0,2649,0.08953975,1.736402,0.6644017,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA
7,4945,1,17,17,7.0,174,0.10457516,1.843137,0.7413793,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA
7,8277,1,17,17,3.0,2769,0.11557377,1.843033,0.7428675,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA
7,11520,1,17,17,9.0,1285,0.12275986,1.881720,0.7657588,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA
7,13198,1,17,17,1.0,399,0.08539945,1.705234,0.6416040,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,1
7,17638,1,17,17,8.0,71,0.17241379,1.913793,0.7464789,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,1
7,27344,1,17,17,11.0,1356,0.08861789,1.723577,0.6563422,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA
7,32177,1,17,17,10.0,337,0.05993691,1.492114,0.4629080,...,0.8181818,11,0.55,525192,train,6,0.05,3,0.25,NA


In [56]:
train <- as.data.frame(data[data$eval_set == "train",])

In [57]:
train <- as.data.frame(data[data$eval_set == "train",])
train$eval_set <- NULL
train$user_id <- NULL
train$product_id <- NULL
train$order_id <- NULL
train$reordered[is.na(train$reordered)] <- 0

In [58]:
train

up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_orders,user_period,user_mean_days_since_prior,user_total_products,user_reorder_ratio,user_distinct_products,user_average_basket,time_since_last_order,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,reordered
1,17,17,6.0,2649,0.08953975,1.736402,0.6644017,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,7.0,174,0.10457516,1.843137,0.7413793,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,3.0,2769,0.11557377,1.843033,0.7428675,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,9.0,1285,0.12275986,1.881720,0.7657588,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,1.0,399,0.08539945,1.705234,0.6416040,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,1
1,17,17,8.0,71,0.17241379,1.913793,0.7464789,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,1
1,17,17,11.0,1356,0.08861789,1.723577,0.6563422,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,10.0,337,0.05993691,1.492114,0.4629080,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,4.0,22,0.10526316,1.894737,0.7727273,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,0
1,17,17,5.0,141,0.20535714,2.035714,0.8226950,20,203,10.68421,11,0.8181818,11,0.55,6,0.05,3,0.25,1
